#### torchtext 라이브러리로 텍스트 분류 <hr>
- [1]단계 - 데이터 전처리 : 숫자 형식으로 변환하는 것 까지
- [2]단계 - 모델 구현

### Torchtext  <hr>
- 1. 데이터 전처리
- 1-1. 데이터 준비 : 내장 데이터셋 활용
    - AG_NEWS 데이터셋 반복자: label + sentence 튜플형태

In [40]:
import torch
from torchtext.datasets import AG_NEWS

# DataPipe type => iterator typecast
train_iter = iter(AG_NEWS(split='train'))

In [41]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [42]:
import torchtext

torchtext.__version__

'0.17.2'

#### 2. 데이터 처리 파이프라인 준비 <hr>

- 어휘집(Vocab), 단어 벡터(word vector), 토크나이저(tokenizer)
- 가공되지 않은 텍스트 문자열에 대한 데이터 처리 빌딩 블록
- 일반적인 NLP 데이터 처리
    - 1. 가공되지 않은 학습 데이터셋으로 어휘집 생성-> 토큰 목록 또는 반복자를 받는 내장 팩토리 함수(factory function); ``build_vocab_from_iterator()``
    - 2. 사용자는 어휘집에 추가할 특수 기호(special symbols) 전달 가능

In [43]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# tokenizer generate
tokenizer = get_tokenizer('basic_english')

# news train data extraction
train_iter = AG_NEWS(split='train')

In [44]:
## Token generator: tokenize data extracted from training

def yield_tokens(data_iter): # 생성자를 만들고 나서 줘야 돼
    for _, text in data_iter:
        #label, text --> text tokenization
        yield tokenizer(text)

In [45]:
# vocab create
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials= ['<unk>'])

# set <unk> index as 0 
vocab.set_default_index(vocab['<unk>'])

In [46]:
vocab(['<unk>','here', 'is','an', 'example'])


[0, 475, 21, 30, 5297]

In [47]:
## text -> integer encode

text_pipeline = lambda x : vocab(tokenizer(x))

#lables -> integer encode
label_pipeline = lambda x: int(x) -1

[ 3 ] 데이터 배치(batch)와 반복자 생성하기 <hr>

- torch. utils.dataLoader: getitem(), len() 구현한 맵 형태(map-style)
- collate_fn(): DataLoader로부터 생성된 샘플 배치 함수
    - 입력: DataLoader에 배치 크기(batch_size)가 있는 배치(batch)데이터


In [48]:
from torch.utils.data import DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#배치 크기만큼 데이터셋 반환 함수
def collate_batch(batch):
    # 배치 크기만큼의 라벨, 텍스트, 오프셋 값 저장 변수
    label_list, text_list, offsets = [], [], [0]

    #하나씩 뉴스 기사, 라벨 추출 후 저장
    for (_label, _text) in batch:

        #라벨 인코딩, 저장
        label_list.append(label_pipeline(_label))
        
        # 텍스트 인코딩 후 저장
        processed_text = torch.tensor(text_pipeline(_text), dtype= torch.int64)
        text_list.append(processed_text)
        
        #텍스트 오프셋(텍스트 크기 / 길이) 저장
        offsets.append(processed_text.size(0))

    # 텐서화 진행
    label_list = torch.tensor(label_list, dtype= torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0) #cumsum = 누적 합. 
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

In [49]:
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, 
                        batch_size= 8, 
                        shuffle = False, 
                        collate_fn=collate_batch)

In [50]:
# q분류 클래스 수 와 단어사전 갯수
num_classes = len(set([label for (label,text) in train_iter]))
vocab_size = len(vocab)

print(f'vocab_size = {vocab_size}, num_classes = {num_classes}')       

vocab_size = 95811, num_classes = 4


In [51]:
for labels, texts, offsets in dataloader:
    print(labels, texts, offsets)
    break

tensor([2, 2, 2, 2, 2, 2, 2, 2]) tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
           14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
            8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
          294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
            1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
           27,    14,    27,   

In [52]:
# 모델 설계
import torch.nn as nn
# 모델 설계 
# 입력층 : EmbeddingBag Layer = 레이어와 분류 목적을 위한 선형 레이어, 텍스트의 길이는 오프셋
# 은닉층 : Linear - 4개 클래스 분류
class TextModel(nn.Module) :
    def __init__(self, VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS) :
        super().__init__()
        # 모델 구성 층 정의
        self.embedding = nn.EmbeddingBag(VOCAB_SIZE, EMBEDD_DIM, sparse = False)
        self.fc = nn.Linear(EMBEDD_DIM, NUM_CLASS)
        self.init_weights()

    # 가중치 초기화
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    #순방향 학습 진행
    def forward(self, text, offsets) :
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [53]:
#PARAMETERS
HIDDEN_SIZE = 3
EMBEDD_DIM = 64
VOCAB_SIZE = len(vocab)
NUM_CLASS = len(set([label for label, _ in train_iter]))
EPOCHS = 10
LR = 5
BATCH_SIZE = 64


In [54]:
# instance
import  torch.optim as optim
MODEL = TextModel(VOCAB_SIZE, EMBEDD_DIM, HIDDEN_SIZE, NUM_CLASS).to(device)

CRITERION = nn.CrossEntropyLoss()
OPTIMIZER = torch.optim.SGD(MODEL.parameters(), lr=LR)
SCHEDULER = optim.lr_scheduler.StepLR(OPTIMIZER, step_size=1, gamma=0.1) #learning rate decresing



In [55]:
# #학습 관련 변수
# total_acc, total_count = 0,0
# log_interval = 300

def train(model, dataloader, optimizer, criterion, epoch):
    
    model.train()
    
    # 학습 평가 관련 변수들
    total_acc, total_count = 0,0
    log_interval=300
    
    for idx, (label, text, offsets) in enumerate(dataloader):
        predicted_label = model(text, offsets)
        optimizer.zero_grad()
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        
        # 배치 학습 평가
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        
        if idx % log_interval == 0 and idx > 0:
            print(f"epoch : {epoch} batch : {idx} loss : {loss.item()}")
            print(f"Accuracy : {total_acc/total_count}")
            total_acc, total_count = 0,0
            
            break

In [62]:
def evaluate(model, dataloader, criterion):
    model.eval()

    total_acc,total_count = 0,0
    
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            total_acc+= (predicted_label.argmax(1)==label).sum().item()
            total_count +=label.size(0)

    return total_acc/total_count

In [63]:
def predict(model, text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()+1

In [64]:
# 학습

for epoch in range(1, EPOCHS+1):
    train(MODEL, dataloader, OPTIMIZER, CRITERION, epoch)
    acc_val = evaluate(MODEL, dataloader, CRITERION)
    print(f'epoch : {epoch} accuracy : {acc_val}')
    SCHEDULER.step()
    

epoch : 1 batch : 300 loss : 0.40215909481048584
Accuracy : 0.7757475083056479
epoch : 1 accuracy : 0.6413333333333333
epoch : 2 batch : 300 loss : 0.37468504905700684
Accuracy : 0.8197674418604651
epoch : 2 accuracy : 0.6853583333333333
epoch : 3 batch : 300 loss : 0.3683914542198181
Accuracy : 0.8268272425249169
epoch : 3 accuracy : 0.682725
epoch : 4 batch : 300 loss : 0.36846038699150085
Accuracy : 0.8359634551495017
epoch : 4 accuracy : 0.68195
epoch : 5 batch : 300 loss : 0.3684725761413574
Accuracy : 0.8359634551495017
epoch : 5 accuracy : 0.681825
epoch : 6 batch : 300 loss : 0.3684740662574768
Accuracy : 0.8363787375415282
epoch : 6 accuracy : 0.6817916666666667
epoch : 7 batch : 300 loss : 0.3684745132923126
Accuracy : 0.8363787375415282
epoch : 7 accuracy : 0.6817916666666667
epoch : 8 batch : 300 loss : 0.3684743046760559
Accuracy : 0.8363787375415282
epoch : 8 accuracy : 0.6817916666666667
epoch : 9 batch : 300 loss : 0.3684743046760559
Accuracy : 0.8363787375415282
epoch 